In [1]:
from helper.analysis import MyDoc
from pathlib import Path
import spacy
import pickle

In [2]:
CORPORA = Path(r"./Corpora/")
PICKLE = Path(r"./auxfiles/pickle/")
nlp = spacy.load("en_core_web_md")

for translator in ["Quixote", "Ibsen"]:
    path = CORPORA/f"Proc_{translator}" 
    docs = [MyDoc(file, nlp) for file in path.iterdir() if file.suffix == ".txt"]
    doc_data = pickle.dumps(docs)
    with open(PICKLE/f"{translator}.pickle", "wb") as f:
        f.write(doc_data)

If we were to pick up the process from this point, we can load the pickle file from disk.

In [ ]:
# with open(PICKLE/"Quixote.pickle", "rb") as f:
#     doc_data=f.read()

# docs = pickle.loads(doc_data)